In [1]:
import numpy as np
import numpy.linalg as LA
import meshplot as mp
from IPython.display import JSON as DJSON
from IPython.display import clear_output
from pspart import Part
from pspart import NNHash
import os
import pandas as ps
from mate_proposals import mate_proposals, homogenize_frame
from scipy.spatial.transform import Rotation as R
import meshplot as mp
import onshape.brepio as brepio

In [2]:
datapath = '/projects/grail/benjones/cadlab'

In [3]:
name = '/fast/jamesn8/assembly_data/assembly_data_with_transforms_all.h5'
assembly_df = ps.read_hdf(name,'assembly')
mate_df = ps.read_hdf(name,'mate')
part_df = ps.read_hdf(name,'part')
mate_df['MateIndex'] = mate_df.index

In [4]:
mate_df.set_index('Assembly', inplace=True)
part_df.set_index('Assembly', inplace=True)    

In [5]:
with open('fully_connected_moving_no_multimates.txt','r') as f:
    set_E_indices = [int(l.rstrip()) for l in f.readlines()]

In [39]:
epsilon_rel = 0.001

all_stats = []
processed_indices = []
for ind in set_E_indices[:10]:
    stats = dict()
    #clear_output(wait=True)
    #display(f'num_processed: {ind}/{assembly_df.shape[0]}')

    #1. spatially hash all MCFs (cache hash maps for each part for re-use with individual mates)
    #2. for all mates, ensure that each MCF is represented (keep track of closest/equivalent MCFs, log percentage of assemblies for which this holds)
    #3. get proposals, edit appropriate ones to true based on equivalence class computed per mated pair of parts (taking outer product of equivalent MCs on left and right)

    print(f'assembly {assembly_df.loc[ind,"AssemblyPath"]}')
    
    part_subset = part_df.loc[ind]
    mate_subset = mate_df.loc[ind]
    if mate_subset.ndim == 1:
        mate_subset = ps.DataFrame([mate_subset], columns=mate_subset.keys())
    
    parts = []
    transforms = []
    mcf_hashes = []
    mco_hashes = []
    mc_frames_all = []
    occ_to_index = dict()
    
    #debug
    #all_points = []
    
    for j in range(part_subset.shape[0]):
        path = os.path.join(datapath, 'data/models', *[part_subset.iloc[j][k] for k in ['did','mv','eid','config']], f'{part_subset.iloc[j]["PartId"]}.xt')
        assert(os.path.isfile(path))
        part = Part(path)
        parts.append(part)
        tf = part_subset.iloc[j]['Transform']
        transforms.append(tf)
        occ_to_index[part_subset.iloc[j]['PartOccurrenceID']] = j
        
    maxdim = max([(part.bounding_box()[1]-part.bounding_box()[0]).max() for part in parts])
    threshold = maxdim * epsilon_rel
    
    for j in range(len(parts)):
        part = parts[j]
        tf = transforms[j]
        mc_frames = []
        mc_origins = []
        for mc in part.all_mate_connectors:
            cs = mc.get_coordinate_system()
            frame = tf[:3,:3] @ cs[:3,:3]
            frame_homogenized = homogenize_frame(frame, z_flip_only=True)
            origin = tf[:3,:3] @ cs[:3,3] + tf[:3,3]
            #all_points.append(origin)
            rot = R.from_matrix(frame_homogenized).as_quat()
            mc_origins.append(origin)
            mc_frames.append(np.concatenate([origin/maxdim, rot]))
        mc_frames_all.append(mc_frames)
        frame_hash = NNHash(mc_frames, 7, epsilon_rel)
        origin_hash = NNHash(mc_origins, 3, threshold)
        #frame_hash = NNHash([mc_frame[:3] for mc_frame in mc_frames], 3, threshold)
        mcf_hashes.append(frame_hash)
        mco_hashes.append(origin_hash)
    
    stats['invalid_frames'] = 0
    stats['invalid_mates'] = 0
    stats['invalid_coincident_origins'] = 0
    stats['invalid_permuted_z'] = 0
    stats['total_mates'] = mate_subset.shape[0]
    stats['maxdim'] = maxdim
    
    #all_points = np.array(all_points)
    #p = mp.plot(all_points)
    for j in range(mate_subset.shape[0]):
        mate_invalid = False
        for i in range(2):
            occId = mate_subset.iloc[j][f'Part{i+1}']
            partIndex = occ_to_index[occId]
            assert(part_subset.iloc[partIndex]['PartOccurrenceID'] == occId)
            origin_local = mate_subset.iloc[j][f'Origin{i+1}']
            frame_local = mate_subset.iloc[j][f'Axes{i+1}']
            tf = transforms[partIndex]
            origin = tf[:3,:3] @ origin_local + tf[:3,3]            
            frame = tf[:3,:3] @ frame_local
            frame_homogenized = homogenize_frame(frame, z_flip_only=True)
            rot = R.from_matrix(frame_homogenized).as_quat()
            mc_frame = np.concatenate([origin/maxdim, rot])
            neighbors = mcf_hashes[partIndex].get_nearest_points(mc_frame)
            
            if len(neighbors) == 0:
                stats['invalid_frames'] += 1
                if not mate_invalid:
                    stats['invalid_mates'] += 1
                mate_invalid = True
                origin_neighbors = mco_hashes[partIndex].get_nearest_points(origin)
                if len(origin_neighbors) > 0:
                    stats['invalid_coincident_origins'] += 1
                    n = next(iter(origin_neighbors))
                    c_frame = R.from_quat(mc_frames_all[partIndex][n][3:]).as_matrix()
                    c_frame_homogenized = homogenize_frame(c_frame, z_flip_only=False)
                    mate_frame_homogenized = homogenize_frame(frame, z_flip_only=False)
                    dist = LA.norm(c_frame_homogenized - mate_frame_homogenized)
                    if dist < threshold:
                        stats['invalid_permuted_z'] += 1
                    
#             #neighbors = mcf_hashes[occ_to_index[occId]].get_nearest_points(origin)
#             print(f'{i} closest dist (fraction of maxdim): {minDist/maxdim}')
#             invalid = dict()
#             invalid['mc_origin'] = mc_frames_all[partIndex][k][:3]
#             invalid['mc_frame'] = R.from_quat(mc_frames_all[partIndex][k][3:]).as_matrix() 
#             invalid['mate_origin'] = origin
#             invalid['mate_frame'] = frame_homogenized
#             invalid['maxdim'] = maxdim
#             if not found:
#                 num_invalid_mates += 1
#                 invalids.append(invalid)
#                 #p.add_points(origin[np.newaxis],shading={'point_color':'blue','point_size':0.02})
#                 print(f'invalid mate {mate_subset.iloc[j]["MateIndex"]}') 
#                 break
#             else:
#                 valids.append(invalid)
    all_stats.append(stats)
    processed_indices.append(ind)
stats_df = ps.DataFrame(all_stats, index=processed_indices)

assembly 0e2de88aa724d660e4093564_52ea6bfdd317472a99efcffa_316901caae65ff373c71ef88
assembly 20acdbfb860686db5801f7db_5676ec505c49f9284cf6f810_69f3bde151e038c3d9a8d55c
assembly 7886c69b1f149069e7a43bdb_b809b448b6da81db4b2388a0_1dc646f33c96254f526ea650
assembly e04c8a49d30adb3a5c0f1deb_3d9b45359a15b248f75e41a2_070617843f30f132ab9e6661
assembly 2a9129b6c469ef81930627d8_51e1cbb836847c0b96790da2_e1f1943b65033b76f7434e20
assembly 571ffca7e4b0e6559901315e_bfd7d4f20f860739096330d1_98ad8c8c79c4beba764ec80e
assembly d8a12ba89bdafc500aff51b8_a249df9d9e0cc2c14fe98430_2f63c6ac02f4bb95b19ad6b4
assembly 5e59e0a31e2d063179d0ef27_1266e2d5d425f14833ba440e_b18634310f6afcbec59c3f10
assembly e4803faed1b9357f8db3722c_ce43730c0f1758f756fc271f_c00b5256d7e874e534c083e8
assembly 080bda692c7362d9d8f550c0_2bb4d493fe1adaf882bc9c9f_2b9ddd420bbfde38934d34ef


In [40]:
stats_df

,invalid_frames,invalid_mates,invalid_coincident_origins,invalid_permuted_z,total_mates,maxdim
18,11,9,1,0,17,0.274000
19,0,0,0,0,2,2.159000
22,1,1,0,0,21,0.200000
23,0,0,0,0,2,0.414338
26,7,7,0,0,15,0.015000
38,0,0,0,0,6,0.228600
40,0,0,0,0,1,0.023094
45,0,0,0,0,3,0.114300
53,2,2,0,0,4,0.189050
64,0,0,0,0,42,0.127000


In [38]:
invalid_counts_old

[(9, 17),
 (0, 2),
 (1, 21),
 (0, 2),
 (7, 15),
 (0, 6),
 (0, 1),
 (0, 3),
 (2, 4),
 (0, 42)]

In [33]:
invalid_counts_old = invalid_counts

In [13]:
from visualize import add_axis

In [26]:
invalid = invalids[6]
p = mp.plot(np.vstack([invalid['mc_origin'], invalid['mate_origin']]))
add_axis(p, invalid['mc_origin'], invalid['mc_frame'][:,0], invalid['mc_frame'][:,1], invalid['mc_frame'][:,2], scale=invalid['maxdim']/10)
add_axis(p, invalid['mate_origin'], invalid['mate_frame'][:,0], invalid['mate_frame'][:,1], invalid['mate_frame'][:,2], scale=invalid['maxdim']/10)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.057504…

In [ ]:
list(mate_subset['MateIndex'])

[376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392]

In [96]:
assembly_df[assembly_df['AssemblyPath'] == '0e2de88aa724d660e4093564_52ea6bfdd317472a99efcffa_316901caae65ff373c71ef88']

,AssemblyPath,ConnectedComponents,RigidPieces,LonePieces,NumParts,NumBinaryPartMates,TotalMates
18,0e2de88aa724d660e4093564_52ea6bfdd317472a99efc...,1,10,0,18,17,17


In [99]:
list(mate_df.loc[18,'MateIndex'])

[376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392]